# Lead Detection Inference Template

# Imports

In [1]:
#!export CUDA_VISIBLE_DEVICES=1

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from tqdm import tqdm
import cv2

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import time
import pandas as pd

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')

tf.__version__


## Env setup

# This is needed to display the images.
%matplotlib inline

## Object detection imports
# Here are the imports from the object detection module.

from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

In [2]:
# Paths
root_dir = os.getcwd()
FROZEN_GRAPH_DIR='inference_graph/V-May-03-Checkpoint-Ausgabe-1'
FROZEN_GRAPH_DIR='inference_graph/V-May-04-Checkpoint-Ihr-2'
FROZEN_GRAPH_DIR='inference_graph/V-May-01-Checkpoint-bezahlen-1'
FROZEN_GRAPH_DIR='inference_graph/V-May-05-Checkpoint-Ihr-4'
FROZEN_GRAPH_DIR='inference_graph/V-Jun-04-Checkpoint-Ihr-7'
FROZEN_GRAPH_DIR='inference_graph/V-Jun-05-Checkpoint-Sie-1'
FROZEN_GRAPH_DIR='inference_graph/V-Jun-16-Checkpoint-Geschlafen-1'
FROZEN_GRAPH_DIR='inference_graph/V-Jun-16-Checkpoint-Geschlafen-1-200000'
FROZEN_GRAPH_DIR='inference_graph/V-Jun-23-Checkpoint-Mantell-2-150k'
FROZEN_GRAPH_DIR='inference_graph/V-Jun-25-Checkpoint-Mantell-6-600K'
FROZEN_GRAPH_DIR='inference_graph/V-Jun-25-Checkpoint-Mantell-7-600K'
FROZEN_GRAPH_DIR='inference_graph/V-Jun-29-Checkpoint-Mantell-9-600K'
FROZEN_GRAPH_DIR='inference_graph/V-Jun-30-Checkpoint-Mantell-13-300k'
FROZEN_GRAPH_DIR='inference_graph/V-Jul-11-Checkpoint-Hemd-5'
FROZEN_GRAPH_DIR='inference_graph/V-Jul-11-Checkpoint-Hemd-3'
FROZEN_GRAPH_DIR='inference_graph/V-Jul-12-Checkpoint-Hemd-6'
FROZEN_GRAPH_DIR='inference_graph/V-Jul-14-Checkpoint-Hemd-8'




# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = os.path.join(root_dir, FROZEN_GRAPH_DIR, "frozen_inference_graph.pb")

# List of the strings that is used to add correct label for each box.
# PATH_TO_LABELS = os.path.join(root_dir, 'training_ocr_ssd_fpn_efficientnet', 'object-detection.pbtxt')
PATH_TO_LABELS = os.path.join('data', 'pintype_detection_label_map.pbtxt')
PATH_TO_LABELS = os.path.join('data', 'pinarray_detection_label_map.pbtxt')
PATH_TO_LABELS = os.path.join('data', 'pinonly_detection_label_map.pbtxt')
PATH_TO_LABELS = os.path.join('data', 'lead_detection_label_map.pbtxt')

print("Pb:\t{}\nLabelmap:\t{}".format(PATH_TO_FROZEN_GRAPH, PATH_TO_LABELS))

Pb:	D:\FZ_WS\JyNB\TF_Research_Api_LD_2_0\research\object_detection\inference_graph/V-Jul-14-Checkpoint-Hemd-8\frozen_inference_graph.pb
Labelmap:	data\lead_detection_label_map.pbtxt


## Load a (frozen) Tensorflow model into memory.

In [3]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    print('Graph import successful')

Graph import successful


## Loading label map
Label maps map indices to category names

In [4]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
print(category_index)

{1: {'id': 1, 'name': 'BODY'}, 2: {'id': 2, 'name': 'PIN'}}


## Helper code

In [5]:
def load_image_into_numpy_array(image, grayscale = False):
  (im_width, im_height) = image.size
    
  if 0:
    im_width = 400 if (im_width>400) else im_width
    image = image.crop((0, 0, im_width, im_height))

  if grayscale:
      return np.repeat(np.asarray(image)[:, :, np.newaxis], 3, axis = 2)
  else:
      return np.array(image.getdata()).reshape(
          (im_height, im_width, 3)).astype(np.uint8)

def createDirirectory(path):
    if not os.path.isdir(path):
        os.makedirs(path)
        print ("Creating {}".format(path))
    return path

# Detection

In [7]:
import glob

##########################
# Input
##########################
# Paths
# PATH_TO_IMAGES_DIR = "images/H_Dataset_03/Mixed"
# PATH_TO_IMAGES_DIR = "images/H_Dataset_05/pintype"
# PATH_TO_IMAGES_DIR = "images/H_Dataset_07/extracted/MERGED"
PATH_TO_IMAGES_DIR = "images/H_Dataset_08/H_Dataset_02_PinArray/QFP-Grayscale-ZL"
PATH_TO_IMAGES_DIR = "images/H_Dataset_08/H_Dataset_02_PinOnly/QFP+SOIC-Multiline"
PATH_TO_IMAGES_DIR = "images/H_Dataset_08/H_Dataset_02_PinArray/QFP+SOIC"
PATH_TO_IMAGES_DIR = "D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinOnly/QFP+SOIC-Fragmented"
PATH_TO_IMAGES_DIR = "D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_09/DB_09_V03/DB_Updater"

PATH_TO_TEST_IMAGES_DIR='D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinOnly/QFP-Grayscale-ZL/txt_pin_pinarray_sep_randomize'
PATH_TO_TEST_IMAGES_DIR='D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinOnly/QFP+SOIC-Multiline/test'
PATH_TO_TEST_IMAGES_DIR='D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinOnly/QFP-Grayscale-ZL/txt_pin_pinarray_sep_randomize_p00_s00'
PATH_TO_TEST_IMAGES_DIR = os.path.join(PATH_TO_IMAGES_DIR, "SOT")
PATH_TO_TEST_IMAGES_DIR = os.path.join(PATH_TO_IMAGES_DIR, "train")
PATH_TO_TEST_IMAGES_DIR = os.path.join(PATH_TO_IMAGES_DIR, "test")

##########################
# Output
##########################
# PATH_TO_OUT_IMG_DIR = createDirirectory("debug/images_out-V-13-gekochtes-25k")
# PATH_TO_OUT_IMG_DIR = createDirirectory("debug/images_out-V-May-03-Checkpoint-Ausgabe-1")
# PATH_TO_OUT_IMG_DIR = createDirirectory("debug/images_out-V-May-04-Checkpoint-Ihr-2")
# PATH_TO_OUT_IMG_DIR = createDirirectory("debug/images_out-V-May-01-Checkpoint-bezahlen-1")
# PATH_TO_OUT_IMG_DIR = createDirirectory("debug/images_out-V-May-04-Checkpoint-Ihr-3")
# PATH_TO_OUT_IMG_DIR = createDirirectory("debug/images_out-V-May-05-Checkpoint-Ihr-4")
# PATH_TO_OUT_IMG_DIR = createDirirectory("debug/images_out-V-May-05-Checkpoint-Ihr-7")
PATH_TO_OUT_IMG_DIR = createDirirectory("debug/images_out-V-Jul-14-Checkpoint-Hemd-8")
PATH_TO_OUT_TEST_IMAGES_DIR = os.path.join(PATH_TO_OUT_IMG_DIR, "SOT")
PATH_TO_OUT_TEST_IMAGES_DIR = os.path.join(PATH_TO_OUT_IMG_DIR, "train")
PATH_TO_OUT_TEST_IMAGES_DIR = os.path.join(PATH_TO_OUT_IMG_DIR, "test")

# Image Path
TEST_IMAGE_PATHS = [f for f in glob.glob(PATH_TO_TEST_IMAGES_DIR + "/*.jpg", recursive=True)]

if (len(TEST_IMAGE_PATHS) >0) :
    createDirirectory(PATH_TO_OUT_IMG_DIR)
    createDirirectory(PATH_TO_OUT_TEST_IMAGES_DIR)

# Size, in inches, of the output images.
IMAGE_SIZE = (20, 10)
print("Count TEST_IMAGE_PATHS:\t{}".format(len(TEST_IMAGE_PATHS)))
print("DONE!")

Creating debug/images_out-V-Jul-14-Checkpoint-Hemd-8\test
Count TEST_IMAGE_PATHS:	30
DONE!


In [8]:
def run_inference_for_images(images_path, graph):
    with graph.as_default():
        #config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True,visible_device_list='1'))
        config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
        with tf.Session(config = config) as sess:
            output_dict_array = []
            
            for image_path in tqdm(images_path):
                
                #Read images
                image = Image.open(image_path).convert("RGB")
                
                #Crop the first 400 pixel
                if 0:
                    w, h = image.size
                    w = 400 if (w>400) else w
                    image = image.crop((0, 0, w, h))
                
                
                # Get handles to input and output tensors
                ops = tf.get_default_graph().get_operations()
                all_tensor_names = {output.name for op in ops for output in op.outputs}
                tensor_dict = {}
                for key in [
                    'num_detections', 'detection_boxes', 'detection_scores',
                    'detection_classes', 'detection_masks'
                ]:
                    tensor_name = key + ':0'
                    if tensor_name in all_tensor_names:
                        tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                            tensor_name)
                if 'detection_masks' in tensor_dict:
                    detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                    detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                    # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                    real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                    detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                    detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                        detection_masks, detection_boxes, image.shape[0], image.shape[1])
                    detection_masks_reframed = tf.cast(
                        tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                    # Follow the convention by adding back the batch dimension
                    tensor_dict['detection_masks'] = tf.expand_dims(
                        detection_masks_reframed, 0)
                image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
                
                # Run inference
                output_dict = sess.run(tensor_dict,
                                       feed_dict={image_tensor: np.expand_dims(image, 0)})

                # all outputs are float32 numpy arrays, so convert types as appropriate
                output_dict['num_detections'] = int(output_dict['num_detections'][0])
                output_dict['detection_classes'] = output_dict[
                    'detection_classes'][0].astype(np.uint8)
                output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
                output_dict['detection_scores'] = output_dict['detection_scores'][0]
                if 'detection_masks' in output_dict:
                    output_dict['detection_masks'] = output_dict['detection_masks'][0]

                output_dict_array.append(output_dict)
  
            
    return output_dict_array


In [9]:
# Predition
images = []
print("Running inference sess...")
start = time.time()
output_dict_array = run_inference_for_images(TEST_IMAGE_PATHS, detection_graph)


Running inference sess...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:06<00:00,  4.75it/s]


In [10]:
import pandas as pd
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# # Predition
# images = []
# print("Running inference sess...")
# start = time.time()
# output_dict_array = run_inference_for_images(TEST_IMAGE_PATHS, detection_graph)

# Visualize and Record Prediction Result
print("Running Visualize and Record Prediction Result...")
for index, output_dict in enumerate(tqdm(output_dict_array)):
    # Load Image
    image_path = TEST_IMAGE_PATHS[index]
    image = Image.open(image_path)
    
#     print(image_path)
    try:
        image_np = load_image_into_numpy_array(image, grayscale=False)
    except:
        image_np = load_image_into_numpy_array(image, grayscale=True)
    
    if not (os.path.isdir(PATH_TO_OUT_TEST_IMAGES_DIR)):
        os.makedirs(PATH_TO_OUT_TEST_IMAGES_DIR)
    out_image_path = os.path.join( PATH_TO_OUT_TEST_IMAGES_DIR , os.path.basename(image_path))
    out_text_path = out_image_path.replace('.jpg','.txt')
        
    #     if (output_dict['num_detections']<100):
    #         print(".")
    #         continue
        
    # Output Images
    min_score_thresh=0.6
    vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks'),
          use_normalized_coordinates=True,
          line_thickness=1,
          skip_scores=False,
          skip_labels=True,
        max_boxes_to_draw=256,
        min_score_thresh=min_score_thresh)
    cv2.imwrite(out_image_path, image_np)
#     plt.figure(figsize=IMAGE_SIZE)
#     plt.imshow(image_np)
#     plt.savefig(out_image_path)
#     plt.close("all")

    
    # Output Text
    boxes = output_dict['detection_boxes']
    # get all boxes from an array
    max_boxes_to_draw = boxes.shape[0]
    # get scores to get a threshold
    scores = output_dict['detection_scores']
    # this is set as a default but feel free to adjust it to your needs
    width, height = image.size

    detected_img_list = []
    detected_class_list = []
    detected_score_list = []
    detected_x_min = []
    detected_y_min = []
    detected_x_max = []
    detected_y_max = []

    
    # iterate over all objects found
    for i in range(min(max_boxes_to_draw, boxes.shape[0])):
      if scores is None or scores[i] > min_score_thresh:
          # boxes[i] is the box which will be drawn
          class_name = category_index[output_dict['detection_classes'][i]]['name']
          confidence_score = scores[i]
          y_min = boxes[i][0] * height
          x_min = boxes[i][1] * width
          y_max = boxes[i][2] * height
          x_max = boxes[i][3] * width

          detected_img_list.append(os.path.basename(image_path))
          detected_class_list.append(class_name)
          detected_score_list.append(confidence_score)
          detected_x_min.append(int(x_min))
          detected_y_min.append(int(y_min))
          detected_x_max.append(int(x_max))
          detected_y_max.append(int(y_max))
        
    myPrediction = pd.DataFrame({
        'xmin': detected_x_min,
        'ymin': detected_y_min,
        'xmax': detected_x_max,
        'ymax': detected_y_max,
        'class': detected_class_list,
        'scores': detected_score_list
    })
    myPrediction.to_csv(out_text_path,encoding='utf-8', header=None, index=False, sep=",")
    

print('Done')
end = time.time()
total_time = end - start
print(end - start)

  0%|                                                                                                                                                                                                | 0/30 [00:00<?, ?it/s]

Running Visualize and Record Prediction Result...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:02<00:00, 13.99it/s]

Done
10.145538330078125
